### Job Cancelation

Import the necessary modules and determine how many nodes and cores the allocation contains.

In [ ]:
import time
import flux
from flux.job import JobspecV1

In [ ]:
ncores = !flux resource list -no {ncores} --state=up
nc = int(ncores[0])

nnodes = !flux resource list -no {nnodes} --state=up
nn = int(nnodes[0])

f = flux.Flux()

Create a `sleep` job specifcation that consumes all the cores in the allocation.

In [ ]:
sleep_jobspec = JobspecV1.from_command(
    ["sleep", "60"], num_tasks=nn, cores_per_task=int(nc/nn)
)

Now launch two `sleep` jobs

In [ ]:
first_jobid = flux.job.submit(f, sleep_jobspec, waitable=True)
print("Submitted 1st job: %d" % (int(first_jobid)))

second_jobid = flux.job.submit(f, sleep_jobspec, waitable=True)
print("Submitted 2nd job: %d\n" % (int(second_jobid)))

Note that the second job is in the _scheduled_ state because the first job is consuming all the cores in the allocation.

In [ ]:
jobs = flux.job.JobList(f, max_entries=2).jobs()
print("First submitted job status (%d) - %s" % (int(jobs[1].id.dec), jobs[1].status))
print("Second submitted job status (%d) - %s\n" % (int(jobs[0].id.dec), jobs[0].status))

Use `flux.job.cancel()` to cancel the first job so that the second job can enter the _run_ state.

In [ ]:
flux.job.cancel(f, first_jobid)
future = flux.job.wait_async(f, first_jobid).wait_for(5.0)
return_id, success, errmsg = future.get_status()
print("Canceled first job: %d\n" % (int(return_id)))

In [ ]:
jobs = flux.job.JobList(f, max_entries=2).jobs()
print("First submitted job status (%d) - %s" % (int(jobs[1].id.dec), jobs[1].status))
print("Second submitted job status (%d) - %s" % (int(jobs[0].id.dec), jobs[0].status))